# Naive Bayes Conti

### Import Libraries

In [1]:
import csv
import math
import random
import pandas as pd
import numpy as np

In [2]:
# Load dataset
data = pd.read_csv('diabetes.csv',delimiter = ',')

# print(data)
# print(type(data))
# Extract headers and data
X = data.iloc[:,0:7]
y = data.iloc[:,8]
# print(X)
# print(y)

### Check shape

In [3]:
# check number of samples
m = X.shape[0]
print("Number of samples: ", m)
# check number of features
n = X.shape[1]
print("Number of features: ", n)

# check number of y
assert m == y.shape[0]

Number of samples:  768
Number of features:  7


In [4]:
#Pandas 
#Convert to DataFrame to check for missing data
df = pd.DataFrame(X)
print('Missing values for train data:\n------------------------\n', df.isnull().sum())

Missing values for train data:
------------------------
 Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
dtype: int64


In [5]:
X = df.values
print(X)

[[  6.    148.     72.    ...   0.     33.6     0.627]
 [  1.     85.     66.    ...   0.     26.6     0.351]
 [  8.    183.     64.    ...   0.     23.3     0.672]
 ...
 [  5.    121.     72.    ... 112.     26.2     0.245]
 [  1.    126.     60.    ...   0.     30.1     0.349]
 [  1.     93.     70.    ...   0.     30.4     0.315]]


### Normalize/Standardize

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)
print(X)

[[ 0.63994726  0.84832379  0.14964075 ... -0.69289057  0.20401277
   0.46849198]
 [-0.84488505 -1.12339636 -0.16054575 ... -0.69289057 -0.68442195
  -0.36506078]
 [ 1.23388019  1.94372388 -0.26394125 ... -0.69289057 -1.10325546
   0.60439732]
 ...
 [ 0.3429808   0.00330087  0.14964075 ...  0.27959377 -0.73518964
  -0.68519336]
 [-0.84488505  0.1597866  -0.47073225 ... -0.69289057 -0.24020459
  -0.37110101]
 [-0.84488505 -0.8730192   0.04624525 ... -0.69289057 -0.20212881
  -0.47378505]]


### Split Train/Test

In [7]:
import random
m, n = X.shape
idx = np.arange(0, m)
random.shuffle(idx)
percent_train = 0.6
m_train = int(m * percent_train)
train_idx = idx[0:m_train]
test_idx = idx[m_train:]
X_train = X[train_idx,:];
X_test = X[test_idx,:];

y_train = y[train_idx];
y_test = y[test_idx];

# double check the shapes
assert X_train.shape[0] == y_train.shape[0]
# assert len(X_train)  == len(y_train)
assert X_test.shape[0] == y_test.shape[0]
# assert len(X_test) == len(y_test)

print('shapes of X_train, y_train')
print(X_train.shape,y_train.shape)
print('shapes of X_test, y_test')
print(X_test.shape,y_test.shape)
# print(y)

shapes of X_train, y_train
(460, 7) (460,)
shapes of X_test, y_test
(308, 7) (308,)


### Distribution of each features by class

In [8]:
def get_data_byclass(X,y):
    labels = np.unique(y)
    X_class = {}
    y_class = {}
    for label in labels:
        X_class[label] = X[y==label]
        y_class[label] = y[y==label]
    return X_class,y_class
        
#     X0 = X[y==0,:]
#     X1 = X[y==1,:]
#     y0 = y[y==0]
#     y1 = y[y==1]
#     return X0,X1,y0,y1
# labels = np.unique(y_train)

# X = (X-np.mean(X, axis = 0))/np.std(X, axis = 0)
# print(X)

In [9]:
X_train_byclass,y_train_byclass= get_data_byclass(X_train,y_train)
print(X_train_byclass.keys())
print(X_train_byclass[0].shape)
print(X_train_byclass[1].shape)
print(y_train_byclass.keys())

X_test_byclass,y_test_byclass= get_data_byclass(X_test,y_test)
print(X_test_byclass.keys())
print(X_test_byclass[0].shape)
print(X_test_byclass[1].shape)
print(y_test_byclass.keys())


# print(X_train_byclass.items())

# X0_train,X1_train,y0_train,y1_train = get_data_byclass(X_train,y_train)
# X0_test,X1_test,y0_test,y1_test = get_data_byclass(X_test,y_test)
# print('shapes of X_train, y_train')
# print(X0_train.shape,y0_train.shape)
# print(X1_train.shape,y1_train.shape)
# print('shapes of X_test, y_test')
# print(X0_test.shape,y0_test.shape)
# print(X1_test.shape,y1_test.shape)


dict_keys([0, 1])
(307, 7)
(153, 7)
dict_keys([0, 1])
dict_keys([0, 1])
(193, 7)
(115, 7)
dict_keys([0, 1])


In [10]:
def get_gaussian_params(X_byclass):
    parameters = [(np.mean(attribute),np.std(attribute)) for attribute in zip(*X_byclass)]
#     print(parameters)
######### why -1
#     del parameters[-1]
    return parameters
                
def get_params(X_byclass):
    parameters ={}
    for key,values in X_byclass.items():
        parameters[key] = get_gaussian_params(values)
    return parameters
parameters = get_params(X_train_byclass)
print('='*30)
print('Mean and std of each class')
print(parameters)
print('='*30)
print('Number of std and mean in a class')
print(len(parameters[0]))
print(len(parameters[1]))
print(type(parameters[0]))
# X0_mean,X0_std = get_mean_std(X_train_byclass,y_train_byclass)  
# X1_mean,X1_std = get_mean_std(X1_train)
# print("="*30)
# print('shapes of mean X_train')
# print(X0_mean.shape,X0_std.shape)
# print("="*30)
# print('shapes of mean X_test')
# print(X1_mean.shape,X1_std.shape)

Mean and std of each class
{0: [(-0.1890438100407975, 0.9033326170665503), (-0.3362782532574676, 0.8275804036898107), (0.02805842225426604, 0.8592729145502395), (-0.03855481347210192, 0.914064656765675), (-0.08432421538909939, 0.9042602418746346), (-0.18790724363413805, 0.9861818265977425), (-0.17640665419812773, 0.9076840916092841)], 1: [(0.2672834638042525, 1.1160791238356824), (0.5813775550727414, 1.0271841530484265), (0.0097527234982741, 1.2108609605780802), (0.14264355967463607, 1.0961145955481977), (0.22522635736847216, 1.2064485375695868), (0.48912482774756155, 0.900496120292233), (0.31069571132124324, 1.2543234621418713)]}
Number of std and mean in a class
7
7
<class 'list'>


### Calculate priors

In [11]:
def get_prior(X_byclass,X):
    priors= {}
    num = X.shape[0]
    for key,values in X_byclass.items():
        priors[key] = len(X_byclass[key])/num
    return priors
priors = get_prior(X_train_byclass,X_train)
print(priors)

{0: 0.6673913043478261, 1: 0.33260869565217394}


### Calculate likelihood

In [12]:
def get_gaussian_pdf(X,params):
    gaussian_pdf = {}
    parameters={}
    gaussian_pdf={}
    exponent={}
    for key, values in params.items():#,params.items():
        # parameters[key][0] = mean, parameters[key][1] = std
        parameters[key] = [params for params in zip(*params[key])]
        exponent[key] = np.exp(-(np.power(X-parameters[key][0],2)/(2*np.power(parameters[key][1],2))))
        gaussian_pdf[key] =(1/(np.sqrt(2*np.pi)*np.array(parameters[key][1]))*exponent[key])
    return gaussian_pdf

likelihood_train = get_gaussian_pdf(X_train,parameters)
#likelihood of x belonging to each class [0 = key]
print(len(likelihood_train[0]))
print(len(likelihood_train[1]))
print(len(likelihood_train[0][0])) #same number of features

assert len(likelihood_train[0]) == (X_train.shape[0])
assert len(likelihood_train[0][0]) == (X_train.shape[1])
assert len(likelihood_train[1]) == (X_train.shape[0])
assert len(likelihood_train[1][0]) == (X_train.shape[1])

likelihood_test = get_gaussian_pdf(X_test,parameters)
print(len(likelihood_test[0]))
print(len(likelihood_test[1]))

assert len(likelihood_test[0]) == (X_test.shape[0])
assert len(likelihood_test[0][0]) == (X_test.shape[1])
assert len(likelihood_test[1]) == (X_test.shape[0])
assert len(likelihood_test[1][0]) == (X_test.shape[1])

# likelihood1_train = get_gaussian_pdf(X_train,X1_mean,X1_std)
# print("="*30)
# print('shapes of pop X_train by feature')
# print(likelihood0_train.shape)
# print(likelihood1_train.shape)
# print("="*30)
# print('shapes of pop X_test by feature')
# likelihood0_test = get_gaussian_pdf(X_test,X0_mean,X0_std)
# likelihood1_test = get_gaussian_pdf(X_test,X1_mean,X1_std)
# print(likelihood0_test.shape)
# print(likelihood1_test.shape)

460
460
7
308
308


### Calculate total likelihood by np.prod

In [13]:
def total_likelihood(likelihood):
    tot_likelihood={}
    for key,values in likelihood.items():
        tot_likelihood[key] = np.prod(likelihood[key],axis = 1)
    return tot_likelihood

total_likelihood_train = total_likelihood(likelihood_train)
print(total_likelihood_train[0].shape)
total_likelihood_test = total_likelihood(likelihood_test)
print(total_likelihood_test[0].shape)
# total_likelihood0_train = np.prod(likelihood0_train,axis=1)
# total_likelihood1_train = np.prod(likelihood1_train,axis=1)
# total_likelihood0_test = np.prod(likelihood0_test,axis=1)
# total_likelihood1_test = np.prod(likelihood1_test,axis=1)

# print('shapes of total_likelihood X_train by feature')
# print(total_likelihood0_train.shape)
# print(total_likelihood1_train.shape)
# print('shapes of total_likelihood X_test by feature')
# print(total_likelihood0_test.shape)
# print(total_likelihood1_test.shape)

(460,)
(308,)


### Calculate the posterior

In [14]:
def posterior(prior,total_likelihood):
    posterior = {}
    for key,values in prior.items():
        posterior[key] = prior[key]*total_likelihood[key]
    return posterior

posterior_train = posterior(priors,total_likelihood_train)
# print(posterior_train)
posterior_test = posterior(priors,total_likelihood_test)
# print(posterior_test)

# posterior0_train = prior0_train*total_likelihood0_train
# posterior1_train = prior1_train*total_likelihood1_train

# print(posterior0_train.shape)
# print(posterior1_train.shape)

# posterior0_test = prior0_train*total_likelihood0_test
# posterior1_test = prior1_train*total_likelihood1_test

# print(posterior0_train.shape)
# print(posterior1_train.shape)

### Calculate accuracy

In [15]:
def predict(posterior):
#     predict = []
    class_names = [key for key in posterior.keys()]
    print(class_names)
    
    sample_length = len(posterior[class_names[0]])
    print(sample_length)
    yhat = []
    for i_col in range(sample_length): 
        column_vector = [posterior[class_names][i_col] for class_names in posterior.keys()]
#         print(column_vector)
        yhat.append(np.argmax(column_vector)) 
    return yhat

# yhat = predict(posterior_train)

# print(yhat)
def get_acc(yhat,y):
    return(yhat == y).sum()/len(y)

yhat_train = predict(posterior_train)
yhat_test = predict(posterior_test)
acc_train = get_acc(yhat_train,y_train)
print('yhat_train',len(yhat_train))
print('y_train',len(y_train))
print('yhat_test',len(yhat_test))
print('y_test',len(y_test))
acc_test = get_acc(yhat_test,y_test)
print('yhat_train',len(yhat_test))
print('y_train',len(y_test))
print("accuracy of data train: ",acc_train)
print("accuracy of data test: ",acc_test)


[0, 1]
460
[0, 1]
308
yhat_train 460
y_train 460
yhat_test 308
y_test 308
yhat_train 308
y_train 308
accuracy of data train:  0.8021739130434783
accuracy of data test:  0.711038961038961
